In [67]:
from yahoofinancials import YahooFinancials
from pprint import pprint
from datetime import datetime
from urllib import request
import json
import pickle
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
import pyodbc
from collections import Counter
import re
from nltk.tokenize import word_tokenize
import requests

## Yahoo Financials Data

In [4]:
vis_stocks = ['MSFT', 'ORCL', 'AMZN', 'CRM']

In [5]:
yahoo_financials_vis = YahooFinancials(vis_stocks)

In [6]:
vis_balance_sheet_data = yahoo_financials_vis.get_financial_stmts('annual', 'balance')

In [7]:
amazon_date = list(vis_balance_sheet_data.get('balanceSheetHistory').get('AMZN')[0].keys())[0]
amazon_date = datetime.strptime(amazon_date, '%Y-%m-%d')

In [8]:
crm_date = list(vis_balance_sheet_data.get('balanceSheetHistory').get('CRM')[0].keys())[0]
crm_date = datetime.strptime(crm_date, '%Y-%m-%d')

In [9]:
oracle_date = list(vis_balance_sheet_data.get('balanceSheetHistory').get('ORCL')[0].keys())[0]
oracle_date = datetime.strptime(oracle_date, '%Y-%m-%d')

In [10]:
microsoft_date = list(vis_balance_sheet_data.get('balanceSheetHistory').get('MSFT')[0].keys())[0]
microsoft_date =datetime.strptime(microsoft_date, '%Y-%m-%d')

In [11]:
amazon_balance_data = {'Ticker':list(vis_balance_sheet_data.get('balanceSheetHistory').keys())[2],\
                      'Financials Date': amazon_date,\
                      'Cash':vis_balance_sheet_data.get('balanceSheetHistory').get('AMZN')[0].get('2018-12-31').get('cash'),\
                      'Long Term Debt': vis_balance_sheet_data.get('balanceSheetHistory').get('AMZN')[0].get('2018-12-31').get('longTermDebt'),
                      'Total Liability': vis_balance_sheet_data.get('balanceSheetHistory').get('AMZN')[0].get('2018-12-31').get('totalLiab'),
                      'Total Assets': vis_balance_sheet_data.get('balanceSheetHistory').get('AMZN')[0].get('2018-12-31').get('totalAssets')}


In [12]:
oracle_balance_data = {'Ticker':list(vis_balance_sheet_data.get('balanceSheetHistory').keys())[1],\
                      'Financials Date': oracle_date,\
                      'Cash':vis_balance_sheet_data.get('balanceSheetHistory').get('ORCL')[0].get('2019-05-31').get('cash'),\
                      'Long Term Debt': vis_balance_sheet_data.get('balanceSheetHistory').get('ORCL')[0].get('2019-05-31').get('longTermDebt'),
                      'Total Liability': vis_balance_sheet_data.get('balanceSheetHistory').get('ORCL')[0].get('2019-05-31').get('totalLiab'),
                      'Total Assets': vis_balance_sheet_data.get('balanceSheetHistory').get('ORCL')[0].get('2019-05-31').get('totalAssets')}

In [13]:
microsoft_balance_data = {'Ticker':list(vis_balance_sheet_data.get('balanceSheetHistory').keys())[0],\
                      'Financials Date': microsoft_date,\
                      'Cash':vis_balance_sheet_data.get('balanceSheetHistory').get('MSFT')[0].get('2019-06-30').get('cash'),\
                      'Long Term Debt': vis_balance_sheet_data.get('balanceSheetHistory').get('MSFT')[0].get('2019-06-30').get('longTermDebt'),
                      'Total Liability': vis_balance_sheet_data.get('balanceSheetHistory').get('MSFT')[0].get('2019-06-30').get('totalLiab'),
                      'Total Assets': vis_balance_sheet_data.get('balanceSheetHistory').get('MSFT')[0].get('2019-06-30').get('totalAssets')}

In [14]:
crm_balance_data = {'Ticker':list(vis_balance_sheet_data.get('balanceSheetHistory').keys())[3],\
                      'Financials Date': crm_date,\
                      'Cash':vis_balance_sheet_data.get('balanceSheetHistory').get('CRM')[0].get('2019-01-31').get('cash'),\
                      'Long Term Debt': vis_balance_sheet_data.get('balanceSheetHistory').get('CRM')[0].get('2019-01-31').get('longTermDebt'),
                      'Total Liability': vis_balance_sheet_data.get('balanceSheetHistory').get('CRM')[0].get('2019-01-31').get('totalLiab'),
                      'Total Assets': vis_balance_sheet_data.get('balanceSheetHistory').get('CRM')[0].get('2019-01-31').get('totalAssets')}

{'Ticker': 'CRM',
 'Financials Date': datetime.datetime(2019, 1, 31, 0, 0),
 'Cash': 2669000000,
 'Long Term Debt': 3173000000,
 'Total Liability': 15132000000,
 'Total Assets': 30737000000}

### SQL connection

In [47]:
conn = pyodbc.connect('DSN=brigitte_finance;''Database=finance;''Trusted_Connection=yes;')
cur = conn.cursor()

In [48]:
c = 'create Table Company(company_id INT IDENTITY PRIMARY KEY, company_name VARCHAR(100), Ticker VARCHAR(5), website VARCHAR(100), wiki_page VARCHAR (1000))'
f = 'create Table Financials(financials_id INT IDENTITY PRIMARY KEY, company_id INT FOREIGN KEY REFERENCES Company(company_id), date DATE, cash FLOAT, long_term_debt FLOAT, total_liability FLOAT, total_assets FLOAT)'
t = 'create Table Top_Words_By_Company(topwords_id INT IDENTITY PRIMARY KEY, company_id INT FOREIGN KEY REFERENCES Company(company_id), word1 VARCHAR(30), word2 VARCHAR(30), word3 VARCHAR(30), word4 VARCHAR(30), word5 VARCHAR(30), word6 VARCHAR(30), word7 VARCHAR(30), word8 VARCHAR(30), word9 VARCHAR(30),word10 VARCHAR(30))'

In [52]:
a = 'drop table Company'
cur.execute(a)
conn.commit()

ProgrammingError: ('42S02', "[42S02] [Microsoft][SQL Server Native Client 11.0][SQL Server]Cannot drop the table 'Company', because it does not exist or you do not have permission. (3701) (SQLExecDirectW)")

In [53]:
cur.execute(c)
conn.commit()

In [54]:
cur.execute(f)
conn.commit()

In [55]:
cur.execute(t)
conn.commit()

In [57]:
i1 = 'insert into Company(company_name, Ticker, website, wiki_page) values \
(\'AMAZON\', \'AMZN\', \'https://www.amazon.com\', \'https://en.wikipedia.org/wiki/Amazon_(company)\'),\
(\'Microsoft\', \'MSFT\', \'https://www.microsoft.com/en-gb/\', \'https://en.wikipedia.org/wiki/Microsoft\'),\
(\'Oracle\', \'ORCL\', \'https://www.oracle.com/uk/index.html\', \'https://en.wikipedia.org/wiki/Oracle\'),\
(\'SalesForce\', \'CRM\', \'https://www.salesforce.com\', \'https://en.wikipedia.org/wiki/Salesforce.com\')'

cur.execute(i1)
conn.commit()

In [59]:
i2 = 'insert into Financials(company_id, date, cash, long_term_debt, total_liability, total_assets) values(?,?,?,?,?,?)'
amzn_params = [1, amazon_balance_data.get('Financials Date'),amazon_balance_data.get('Cash'), amazon_balance_data.get('Long Term Debt'), amazon_balance_data.get('Total Liability'), amazon_balance_data.get('Total Assets')]
msft_params = [2, microsoft_balance_data.get('Financials Date'), microsoft_balance_data.get('Cash'), microsoft_balance_data.get('Long Term Debt'), microsoft_balance_data.get('Total Liability'), microsoft_balance_data.get('Total Assets')]
orcl_params = [3, oracle_balance_data.get('Financials Date'),oracle_balance_data.get('Cash'), oracle_balance_data.get('Long Term Debt'), oracle_balance_data.get('Total Liability'), oracle_balance_data.get('Total Assets')]
crm_params = [4, crm_balance_data.get('Financials Date'), crm_balance_data.get('Cash'), crm_balance_data.get('Long Term Debt'), crm_balance_data.get('Total Liability'), crm_balance_data.get('Total Assets')]
cur.execute(i2,amzn_params)
cur.execute(i2, msft_params)
cur.execute(i2, orcl_params)
cur.execute(i2, crm_params)
conn.commit()

In [60]:
i3 = 'insert into Top_Words_By_Company(company_id, word1, word2, word3, word4, word5, word6, word7, word8, word9, word10) values(?,?,?,?,?,?,?,?,?,?,?)'
amzn_words_params = [1, list_top_amzn[0], list_top_amzn[1], list_top_amzn[2], list_top_amzn[3], list_top_amzn[4], list_top_amzn[5], list_top_amzn[6], list_top_amzn[7], list_top_amzn[8], list_top_amzn[9]]
msft_words_params = [2, list_top_msft[0], list_top_msft[1], list_top_msft[2], list_top_msft[3], list_top_msft[4], list_top_msft[5], list_top_msft[6], list_top_msft[7], list_top_msft[8], list_top_msft[9]]
orcl_words_params = [3, list_top_orcl[0], list_top_orcl[1], list_top_orcl[2], list_top_orcl[3], list_top_orcl[4], list_top_orcl[5], list_top_orcl[6], list_top_orcl[7], list_top_orcl[8], list_top_orcl[9]]
crm_words_params = [4, list_top_crm[0], list_top_crm[1], list_top_crm[2], list_top_crm[3], list_top_crm[4], list_top_crm[5], list_top_crm[6], list_top_crm[7], list_top_crm[8], list_top_crm[9]]

cur.execute(i3, amzn_words_params)
cur.execute(i3, msft_words_params)
cur.execute(i3, orcl_words_params)
cur.execute(i3, crm_words_params)
conn.commit()

## WIKI Webscrape

#### Amazon Wiki Webscrape

In [15]:
amzn_url = 'https://en.wikipedia.org/wiki/Amazon_(company)'
h ={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}

In [16]:
req = request.Request(amzn_url, headers = h)
with request.urlopen(req) as r:
    soup = BeautifulSoup(r, 'html.parser')

In [17]:
amazon_parse = soup.findAll('div', {'class':'mw-body-content'})

In [18]:
words = []
for a in amazon_parse:
    words.append(a.text.split())

In [19]:
amzn_words_list = words[2]

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BrigitteKudor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
stop = list(stopwords.words('english'))

In [22]:
amzn_list = []
for word in amzn_words_list:
    w = re.sub(r"[^A-Za-z]","", word)
    if w != '':
        amzn_list.append(w)

In [23]:
def word_cleanser(your_list):
    clean_list = []
    for word in your_list:
        w = re.sub(r"[^A-Za-z]","", word)
        if w != '':
            clean_list.append(w)
    return clean_list

In [24]:
def remove_stopwords(word_list):
        processed_word_list = []
        for word in word_list:
            word = word.lower() # in case they arenet all lower cased
            if word not in stopwords.words("english"):
                processed_word_list.append(word)
        return processed_word_list

In [25]:
amzn_list = remove_stopwords(amzn_list)

In [26]:
def top_10_words(your_list):
    counter = Counter(your_list)
    most_occur = counter.most_common(10)
    return most_occur

In [73]:
def top_words_as_list(your_list):
    just_word = []
    for elem in your_list:
        elem = list(elem)
        just_word.append(elem[0])
    return just_word

In [45]:
top_amzn = top_10_words(amzn_list)
list_top_amzn = top_words_as_list(top_amzn)

In [46]:
list_top_amzn

['amazon',
 'retrieved',
 'amazoncom',
 'february',
 'company',
 'sales',
 'november',
 'september',
 'us',
 'music']

### Microsoft webscrape

In [39]:
msft_url = 'https://en.wikipedia.org/wiki/Microsoft'
h ={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}
req = request.Request(msft_url, headers = h)
with request.urlopen(req) as r:
    soup = BeautifulSoup(r, 'html.parser')

In [40]:
msft_parse =soup.findAll('div', {'id':'bodyContent'}) # soup.findAll('div', {'class':'mw-body-content'})
words_msft = []
for a in msft_parse:
    words_msft.append(a.text.split())
msft_word_list = words_msft[0]

In [41]:
cleaned_msft = word_cleanser(msft_word_list)

In [42]:
cleaned_msft2 = remove_stopwords(cleaned_msft)

In [43]:
top_msft = top_10_words(cleaned_msft2)

In [44]:
list_top_msft = top_words_as_list(top_msft)

### Oracle webscrape

In [28]:
orcl_url = 'https://en.wikipedia.org/wiki/Oracle_Corporation'
h ={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}
req = request.Request(orcl_url, headers = h)
with request.urlopen(req) as r:
    soup = BeautifulSoup(r, 'html.parser')

In [29]:
orcl_parse =soup.findAll('div', {'id':'bodyContent'}) # soup.findAll('div', {'class':'mw-body-content'})
words_orcl = []
for a in orcl_parse:
    words_orcl.append(a.text.split())
orcl_word_list = words_orcl[0]

In [37]:
cleaned_orcl = word_cleanser(orcl_word_list)
cleaned_orcl2 = remove_stopwords(cleaned_orcl)
top_orcl = top_10_words(cleaned_orcl2)
list_top_orcl = top_words_as_list(top_orcl)

### SalesForce webscrape

In [31]:
crm_url = 'https://en.wikipedia.org/wiki/Salesforce.com'
h ={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}
req = request.Request(crm_url, headers = h)
with request.urlopen(req) as r:
    soup = BeautifulSoup(r, 'html.parser')

In [32]:
crm_parse =soup.findAll('div', {'id':'bodyContent'}) # soup.findAll('div', {'class':'mw-body-content'})
words_crm = []
for a in crm_parse:
    words_crm.append(a.text.split())
crm_word_list = words_crm[0]

In [36]:
cleaned_crm = word_cleanser(crm_word_list)
cleaned_crm2 = remove_stopwords(cleaned_crm)
top_crm = top_10_words(cleaned_crm2)
list_top_crm = top_words_as_list(top_crm)

In [ ]:
list_top_crm